In [1]:
#Importing and loading libraries
import numpy as np
import pandas as pd
#For oversampling
from collections import Counter
from imblearn.over_sampling import ADASYN
#Train-test sets splitting
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
#For DecisionTrees
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
#For Logistic Regression classifier
from sklearn.linear_model import LogisticRegression
#For Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
#For Support Vector Machine
from sklearn import svm
#For Random Forest
from sklearn.ensemble import RandomForestClassifier

In [3]:
df = pd.read_csv("C:\\Users\\akram\\Desktop\\Dissertation\\Datasets\\PCDataset.csv")

features = df.drop(['InjuredNextTrainingSession'], axis = 1)
target = df['InjuredNextTrainingSession']

In [4]:
#Looking at proportion of '0's and '1's in the target column
imbal_proportion = Counter(target)
print(imbal_proportion)
#Oversampling the training set
oversample = ADASYN()
featuresi, targeti = oversample.fit_resample(features, target)
#Looking at new proportion of '0's and '1's
bal_proportion = Counter(targeti)
print(bal_proportion)

Counter({0: 3805, 1: 38})
Counter({0: 3805, 1: 3800})


In [5]:
#Function to return accuracy metrics
def accuraciz(y_test, y_pred):
    count = Counter(y_test)
    real_values_zero = count[0]
    real_values_one = count[1]
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    #Transform the data into a numerical vector
    y_test = list(map(int, y_test))
    y_pred = list(map(int, y_pred))
    
    for i in range(len(y_pred)):
        if y_test[i] == 1 and y_pred[i] == 1:
            TP += 1
        if y_pred[i] == 1 and y_test[i] != y_pred[i]:
            FP += 1
        if y_test[i] == y_pred[i] == 0:
            TN += 1
        if y_pred[i] == 0 and y_test[i] != y_pred[i]:
            FN += 1
    
    sensitivity = TP/(TP + FN)
    specificity = TN/(FP + TN)
    FPR = FP / real_values_zero
    FNR = FN / real_values_one
    acc = (TP + TN) / (real_values_one + real_values_zero)
    ba = (sensitivity + specificity) / 2
    f1 = (2*TP) / (2*TP + FP + FN)
    LRplus = sensitivity/(1-specificity)
    LRminus = (1-sensitivity)/specificity
    
    return sensitivity*100, specificity*100, FPR*100, FNR*100, acc*100, ba*100, f1*100, LRplus, LRminus, TP, FP, TN, FN

In [6]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myDT = DecisionTreeClassifier(criterion = 'gini', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myDT = cross_val_predict(myDT, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myDT)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 323390 (42.52333990795529%), FP: 78354 (10.302958579881658%)
TN: 302146 (39.72991452991453%), FN: 56610 (7.443786982248521%)
Sensitivity has mean 85.10263157894738 with variance 1.987001385041553
Specificity has mean 79.40762155059132 with variance 1.1657587274507417
False Positive Rate has mean 20.59237844940867 with variance 1.1657587274507397
False Negative Rate has mean 14.897368421052626 with variance 1.987001385041551
ACC has mean 82.25325443786984 with variance 0.6310202680768606
Balanced Accuracy has mean 82.25512656476934 with variance 0.6312898275559518
F1-Score has mean 82.73204263989018 with variance 0.6899453209123828
Likelihood ratio positive has mean 4.143256918508445 with variance 0.04508040199247661
Likelihood ratio negative has mean 0.1875911479324598 with variance 0.0003025726501615324


In [7]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myDT = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myDT = cross_val_predict(myDT, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myDT)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 321475 (42.2715318869165%), FP: 85568 (11.25154503616042%)
TN: 294932 (38.78132807363576%), FN: 58525 (7.695595003287311%)
Sensitivity has mean 84.59868421052633 with variance 2.8329691828254857
Specificity has mean 77.51169513797635 with variance 1.7159229936403608
False Positive Rate has mean 22.48830486202365 with variance 1.7159229936403626
False Negative Rate has mean 15.401315789473683 with variance 2.8329691828254857
ACC has mean 81.05285996055225 with variance 0.8984063306563708
Balanced Accuracy has mean 81.05518967425134 with variance 0.8987729438328994
F1-Score has mean 81.68685304172215 with variance 0.9867452385165117
Likelihood ratio positive has mean 3.7733955285949237 with variance 0.044742923678186816
Likelihood ratio negative has mean 0.19867320607144257 with variance 0.00045183374311697714


In [8]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Logistic Regression classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myCLR = LogisticRegression(solver='lbfgs', max_iter=1000)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myCLR = cross_val_predict(myCLR, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myCLR)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 227989 (29.978829717291255%), FP: 194212 (25.53740959894806%)
TN: 186288 (24.495463510848126%), FN: 152011 (19.988297172912556%)
Sensitivity has mean 59.99710526315789 with variance 8.273779709141275
Specificity has mean 48.95873850197109 with variance 0.675537996377268
False Positive Rate has mean 51.041261498028916 with variance 0.675537996377268
False Negative Rate has mean 40.0028947368421 with variance 8.273779709141275
ACC has mean 54.47429322813939 with variance 2.0488782035072415
Balanced Accuracy has mean 54.47792188256449 with variance 2.0513749343677397
F1-Score has mean 56.814183022150594 with variance 3.7888716496013344
Likelihood ratio positive has mean 1.1756288060345708 with variance 0.003214025772929973
Likelihood ratio negative has mean 0.8171460459459309 with variance 0.0033802189994064113


In [9]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Naive-Bayes classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myNB = GaussianNB()

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myNB = cross_val_predict(myNB, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myNB)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 300191 (39.47284681130835%), FP: 254352 (33.445364891518736%)
TN: 126148 (16.58750821827745%), FN: 79809 (10.494280078895464%)
Sensitivity has mean 78.99763157894738 with variance 0.09440574792243783
Specificity has mean 33.15321944809462 with variance 0.07558323735454275
False Positive Rate has mean 66.8467805519054 with variance 0.07558323735454238
False Negative Rate has mean 21.00236842105263 with variance 0.09440574792243779
ACC has mean 56.06035502958581 with variance 0.05517631960356919
Balanced Accuracy has mean 56.07542551352098 with variance 0.05518249426170316
F1-Score has mean 64.2432775106793 with variance 0.04127793785369938
Likelihood ratio positive has mean 1.181797407443566 with variance 5.840477681024392e-05
Likelihood ratio negative has mean 0.6335603739027834 with variance 0.00014195441890526184


In [10]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Support Vector classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
mySVC = svm.SVC()

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_mySVC = cross_val_predict(mySVC, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_mySVC)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 300665 (39.53517422748192%), FP: 240609 (31.63826429980276%)
TN: 139891 (18.394608809993425%), FN: 79335 (10.431952662721894%)
Sensitivity has mean 79.12236842105263 with variance 0.37163954293628754
Specificity has mean 36.76504599211563 with variance 0.08382490015040028
False Positive Rate has mean 63.23495400788438 with variance 0.08382490015040003
False Negative Rate has mean 20.877631578947366 with variance 0.3716395429362885
ACC has mean 57.929783037475346 with variance 0.10299573665375551
Balanced Accuracy has mean 57.94370720658414 with variance 0.10309029640000056
F1-Score has mean 65.2707844376775 with variance 0.11463178416146338
Likelihood ratio positive has mean 1.2512649928762165 with variance 0.00011227337018395879
Likelihood ratio negative has mean 0.5678857396231922 with variance 0.0002765369620699598


In [11]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Support Vector classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myRF = RandomForestClassifier(criterion = 'entropy', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myRF = cross_val_predict(myRF, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myRF)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 333415 (43.84155161078238%), FP: 56774 (7.4653517422748195%)
TN: 323726 (42.567521367521366%), FN: 46585 (6.125575279421433%)
Sensitivity has mean 87.74078947368417 with variance 0.48380020775623195
Specificity has mean 85.07910643889619 with variance 0.20998195541173598
False Positive Rate has mean 14.920893561103812 with variance 0.20998195541173603
False Negative Rate has mean 12.25921052631579 with variance 0.4838002077562327
ACC has mean 86.40907297830375 with variance 0.16017731163232576
Balanced Accuracy has mean 86.4099479562902 with variance 0.16026724353768682
F1-Score has mean 86.5791356702072 with variance 0.17761998434758572
Likelihood ratio positive has mean 5.885791297733118 with variance 0.032909809408294535
Likelihood ratio negative has mean 0.14409251990110505 with variance 6.640156651416974e-05
